In [100]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

df = pd.read_csv('../data/text_data.csv')

df

,Тематика,Текст обращения,Количество комментариев,Количество репостов
0,Здравоохранение,NaN,61,50
1,Здравоохранение,Прошу рассмотреть вопрос о ремонте крыши в мно...,92,43
2,Культура,"Транспорт ходит с большими задержками, особенн...",14,43
3,Транспорт,Погибли бездомные животные из-за отравления. К...,94,28
4,Безопасность,Где обещанная модернизация больницы? Оборудова...,77,5
...,...,...,...,...
495,Образование,"Нет воды в доме номер 12 уже вторые сутки, про...",73,44
496,NaN,Прошу рассмотреть вопрос о ремонте крыши в мно...,69,46
497,Культура,Примите меры! В школе не хватает учебников для...,25,45
498,NaN,"Глава региона, прошу рассмотреть вопрос о стро...",38,0


In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Тематика                 479 non-null    object
 1   Текст обращения          460 non-null    object
 2   Количество комментариев  500 non-null    int64 
 3   Количество репостов      500 non-null    int64 
dtypes: int64(2), object(2)
memory usage: 15.8+ KB


In [102]:
df = df.dropna(subset=['Текст обращения']).copy()
df['Тематика'] = df['Тематика'].fillna('Не указано')
df_unique_texts = pd.DataFrame(df["Текст обращения"].dropna().unique(), columns=["Уникальные тексты"])
df_unique_texts

,Уникальные тексты
0,Прошу рассмотреть вопрос о ремонте крыши в мно...
1,"Транспорт ходит с большими задержками, особенн..."
2,Погибли бездомные животные из-за отравления. К...
3,Где обещанная модернизация больницы? Оборудова...
4,"Уважаемая администрация, прошу обратить вниман..."
5,Крик души! В поликлинике номер 5 огромные очер...
6,Примите меры! В школе не хватает учебников для...
7,"Глава региона, прошу рассмотреть вопрос о стро..."
8,"Нет воды в доме номер 12 уже вторые сутки, про..."
9,От лица жителей обращаюсь по поводу отсутствия...


In [103]:
df

,Тематика,Текст обращения,Количество комментариев,Количество репостов
1,Здравоохранение,Прошу рассмотреть вопрос о ремонте крыши в мно...,92,43
2,Культура,"Транспорт ходит с большими задержками, особенн...",14,43
3,Транспорт,Погибли бездомные животные из-за отравления. К...,94,28
4,Безопасность,Где обещанная модернизация больницы? Оборудова...,77,5
6,ЖКХ,"Уважаемая администрация, прошу обратить вниман...",11,3
...,...,...,...,...
495,Образование,"Нет воды в доме номер 12 уже вторые сутки, про...",73,44
496,Не указано,Прошу рассмотреть вопрос о ремонте крыши в мно...,69,46
497,Культура,Примите меры! В школе не хватает учебников для...,25,45
498,Не указано,"Глава региона, прошу рассмотреть вопрос о стро...",38,0


In [104]:
df.duplicated().sum()

np.int64(0)

In [105]:
stop_words = {"и", "в", "во", "не", "что", "он", "на", "я", "с", "со", "как", "а", "то", "все", "она", "так", "его",
                  "но", "да", "ты", "к", "у", "же", "вы", "за", "бы", "по", "только", "ее", "мне", "было", "вот", "от",
                  "меня", "еще", "нет", "о", "из", "ему", "теперь", "когда", "даже", "ну", "вдруг", "ли", "если", "уже",
                  "или", "ни", "быть", "был", "него", "до", "вас", "нибудь", "опять", "уж", "вам", "ведь", "там",
                  "потом", "себя", "ничего", "ей", "может", "они", "тут", "где", "есть", "надо", "ней", "для", "мы",
                  "тебя", "их", "чем", "была", "сам", "чтоб", "без", "будто", "чего", "раз", "тоже", "себе", "под",
                  "будет", "ж", "тогда", "кто", "этот", "того", "потому", "этого", "какой", "совсем", "ним", "здесь",
                  "этом", "почти", "мой", "тем", "чтобы", "нее", "сейчас", "были", "куда", "зачем", "всех", "никогда",
                  "можно", "при", "наконец", "два", "об", "другой", "хоть", "после", "над", "больше", "тот", "через",
                  "эти", "нас", "про", "всего", "них", "какая", "много", "разве", "три", "эту", "моя", "впрочем",
                  "хорошо", "свою", "этой", "перед", "иногда", "лучше", "чуть", "том", "нельзя", "такой", "им", "более",
                  "всегда", "конечно", "всю", "между"}

def clean_text(text):
    text = text.lower()
    text = re.sub(r"\d+", "", text)
    text = re.sub(r"[^\w\s]", "", text)
    words = text.split()
    words = [word for word in words if word not in stop_words]

    return " ".join(words)
df["Текст обращения"] = df["Текст обращения"].apply(clean_text)

In [106]:
vectorizer = TfidfVectorizer(max_features=1000)
X = vectorizer.fit_transform(df["Текст обращения"])

num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters, random_state=143, n_init=10)
df["Кластер"] = kmeans.fit_predict(X)

print(df["Кластер"].value_counts())

df.head()

Кластер
2    59
5    54
3    50
4    48
1    45
7    45
6    44
0    44
8    42
9    29
Name: count, dtype: int64


,Тематика,Текст обращения,Количество комментариев,Количество репостов,Кластер
1,Здравоохранение,прошу рассмотреть вопрос ремонте крыши многокв...,92,43,9
2,Культура,транспорт ходит большими задержками особенно в...,14,43,7
3,Транспорт,погибли бездомные животные изза отравления отв...,94,28,5
4,Безопасность,обещанная модернизация больницы оборудование у...,77,5,0
6,ЖКХ,уважаемая администрация прошу обратить внимани...,11,3,8


In [107]:
for cluster_num in range(num_clusters):
    print(f"\n🔹 Кластер {cluster_num}:")
    print(df[df["Кластер"] == cluster_num]["Текст обращения"].head(5).tolist())


🔹 Кластер 0:
['обещанная модернизация больницы оборудование устарело врачи справляются', 'обещанная модернизация больницы оборудование устарело врачи справляются', 'обещанная модернизация больницы оборудование устарело врачи справляются', 'обещанная модернизация больницы оборудование устарело врачи справляются', 'обещанная модернизация больницы оборудование устарело врачи справляются']

🔹 Кластер 1:
['воды доме номер вторые сутки прошу срочно разобраться', 'воды доме номер вторые сутки прошу срочно разобраться', 'воды доме номер вторые сутки прошу срочно разобраться', 'воды доме номер вторые сутки прошу срочно разобраться', 'воды доме номер вторые сутки прошу срочно разобраться']

🔹 Кластер 2:
['примите меры школе хватает учебников учеников х классов', 'примите меры школе хватает учебников учеников х классов', 'примите меры школе хватает учебников учеников х классов', 'примите меры школе хватает учебников учеников х классов', 'примите меры школе хватает учебников учеников х классов']


In [108]:
cluster_names = {
    0: "Проблемы с больницей",
    1: "Проблемы с водопроводом",
    2: "Нехватка учебников в школе",
    3: "Строительство детской площадки",
    4: "Отсутствие уличного освещения",
    5: "Гибель бездомных животных",
    6: "Очереди в поликлиниках",
    7: "Задержки транспорта",
    8: "Состояние дорог",
    9: "Ремонт крыши"
}

df["Тематика"] = df["Кластер"].map(cluster_names)
df.drop(columns=["Кластер"], inplace=True)
df.head()

,Тематика,Текст обращения,Количество комментариев,Количество репостов
1,Ремонт крыши,прошу рассмотреть вопрос ремонте крыши многокв...,92,43
2,Задержки транспорта,транспорт ходит большими задержками особенно в...,14,43
3,Гибель бездомных животных,погибли бездомные животные изза отравления отв...,94,28
4,Проблемы с больницей,обещанная модернизация больницы оборудование у...,77,5
6,Состояние дорог,уважаемая администрация прошу обратить внимани...,11,3


In [109]:
df

,Тематика,Текст обращения,Количество комментариев,Количество репостов
1,Ремонт крыши,прошу рассмотреть вопрос ремонте крыши многокв...,92,43
2,Задержки транспорта,транспорт ходит большими задержками особенно в...,14,43
3,Гибель бездомных животных,погибли бездомные животные изза отравления отв...,94,28
4,Проблемы с больницей,обещанная модернизация больницы оборудование у...,77,5
6,Состояние дорог,уважаемая администрация прошу обратить внимани...,11,3
...,...,...,...,...
495,Проблемы с водопроводом,воды доме номер вторые сутки прошу срочно разо...,73,44
496,Ремонт крыши,прошу рассмотреть вопрос ремонте крыши многокв...,69,46
497,Нехватка учебников в школе,примите меры школе хватает учебников учеников ...,25,45
498,Строительство детской площадки,глава региона прошу рассмотреть вопрос строите...,38,0


In [110]:
print("Статистика по кластерам:")
print(df["Тематика"].value_counts())

Статистика по кластерам:
Тематика
Нехватка учебников в школе        59
Гибель бездомных животных         54
Строительство детской площадки    50
Отсутствие уличного освещения     48
Проблемы с водопроводом           45
Задержки транспорта               45
Очереди в поликлиниках            44
Проблемы с больницей              44
Состояние дорог                   42
Ремонт крыши                      29
Name: count, dtype: int64


In [111]:
df.to_csv("../data/cleaned_text_data.csv", index=False)